<a href="https://colab.research.google.com/github/AshwinDeshpande96/Speech-Generation/blob/master/president_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from google.colab import drive
from keras.models import load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.optimizers import Adam
import sys
import inflect
import re
import nltk
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer
from nltk.stem import WordNetLemmatizer


Using TensorFlow backend.


In [0]:
nltk.download("wordnet")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [0]:
#drive.mount('/content/gdrive', force_remount=True)
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
president = 'lincoln'
file_path = '/content/gdrive/My Drive/Projects/NLP/President Speech/' +president+'_all.txt'

In [0]:
raw_text = open(file_path).read()

In [0]:
porter = PorterStemmer()
lancaster=LancasterStemmer()
wordnet_lemmatizer = WordNetLemmatizer()

In [0]:
def text_preprocess(raw_text):
    #raw_text = raw_text.lower()
    num_set = re.findall(r'\d+', raw_text)
    p = inflect.engine()
    #print num_set
    for num in num_set:
        word = str(p.number_to_words(num))
        raw_text = raw_text.replace(num, word)
    punctuation_chars = ["!", '"', '&', ',', '?', '/', ':', ';', '<', '>', '$', '#', '@', '%', '*', '(', ')', '[', ']', '{', '}', '\n', '-', '`'] 
    for symbol in punctuation_chars:
        raw_text = raw_text.replace(symbol, ' ')
    raw_text = raw_text.replace('.', ' . ')
    raw_text = raw_text.replace( "'", "" )
    
    
    raw_words = raw_text.split(' ')
    raw_words = numpy.array([str(wordnet_lemmatizer.lemmatize(lancaster.stem(val))) for val in raw_words if val is not ''])
    raw_words
    print "Text Word Count: ", raw_words.shape[0]
    vocab = numpy.unique(sorted(raw_words))
    print "Vocab Length: ", vocab.shape[0]
    return vocab.shape[0],raw_words.shape[0],raw_words, raw_text, vocab
n_vocab, n_words, raw_words, raw_text, vocab = text_preprocess(raw_text)

Text Word Count:  101118
Vocab Length:  3737


In [0]:
#vocab = numpy.delete(vocab, 0)
print "Vocab: ",vocab[:50], "\n...\n", vocab[-10:]
print "Raw Words: ",raw_words[:50], "\n...\n", raw_words[-10:]

Vocab:  ['.' '00seven' '0eight' '0eightsix' '0eighty' '0fivenine' '0fivesix'
 '0fourfour' '0fournine' '0foursix' '0fourthree' '0nineteen' '0oneeight'
 '0onefive' '0oneone' '0seven' '0sevenseven' '0sevensix' '0six' '0sixfive'
 '0sixnine' '0threenine' '0two' '0two0' 'a' 'ab' 'abandon' 'abh' 'abid'
 'abide' 'abl' 'ablest' 'abol' 'abolit' 'aborigin' 'about' 'abov'
 'abraham' 'abridg' 'abroad' 'abrog' 'absolv' 'absorb' 'abstain'
 'abstract' 'absurd' 'abund' 'abus' 'ac' 'academy'] 
...
['yesterday' 'yet' 'yield' 'yo' 'york' 'you' 'young' 'yourselv' 'zeal'
 'zero']
Raw Words:  ['hon' 'to' 'henry' 'clayon' 'the' 'four' 'day' 'of' 'july' 'on'
 'thousand' 'sev' 'hundr' 'and' 'seventy' 'six' 'the' 'peopl' 'of' 'a'
 'few' 'feebl' 'and' 'oppress' 'colony' 'of' 'gre' 'britain' 'inhabit' 'a'
 'port' 'of' 'the' 'atl' 'coast' 'of' 'nor' 'americ' 'publ' 'decl' 'their'
 'nat' 'independ' 'and' 'mad' 'their' 'ap' 'to' 'the' 'just'] 
...
['the' 'right' 'result' '.' 'yo' 'very' 'truly' '.' 'lincoln' '.']


In [0]:
#chars = sorted(list(set(raw_text)))
vocab_to_int = dict((c, i) for i, c in enumerate(vocab))

In [0]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_words - seq_length, 1):
	seq_in = raw_words[i:i + seq_length]
	seq_out = raw_words[i + seq_length]
	dataX.append([vocab_to_int[word] for word in seq_in])
	dataY.append(vocab_to_int[seq_out])
n_patterns = len(dataX)
print "Total Patterns: ", n_patterns

Total Patterns:  101018


In [0]:
print numpy.array(dataX).shape

(114982, 100)


In [0]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
print X.shape



(101018, 100, 1)


In [0]:
# one hot encode the output variable
y = np_utils.to_categorical(dataY)
print y.shape

(101018, 3737)


In [0]:
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dense(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=1.0e-3))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
'''
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
'''

In [0]:
# define the checkpoint
filepath="/content/gdrive/My Drive/Projects/NLP/President Speech/weights/"+president+"_weights.hdf5"

callbacks = [
    EarlyStopping(monitor='loss', patience=10, verbose=0, mode='min'),
    ReduceLROnPlateau(monitor='loss', factor=0.1, patience=5, verbose=1, mode='min'),
    ModelCheckpoint(filepath, save_best_only=True,  save_weights_only=False, monitor='loss', mode='min')
]
#model.load_weights('/content/gdrive/My Drive/Projects/NLP/weights-improvement-20-1.9923.hdf5')

In [0]:
#model = load_model('/content/gdrive/My Drive/Projects/NLP/President Speech/'+president+'_model.hdf5')


In [0]:
#model.fit(X, y, epochs=1, batch_size=1000, callbacks=callbacks)
#model.save('/content/gdrive/My Drive/Projects/NLP/President Speech/'+president+'_model.hdf5')
for i in range(25):
    model = load_model('/content/gdrive/My Drive/Projects/NLP/President Speech/'+president+'_model.hdf5')
    model.fit(X, y, epochs=10, batch_size=64, callbacks=callbacks)
    model.save('/content/gdrive/My Drive/Projects/NLP/President Speech/'+president+'_model.hdf5')

Epoch 1/10
101018/101018 [==============================] - 658s 7ms/step - loss: 6.0508
Epoch 2/10
101018/101018 [==============================] - 643s 6ms/step - loss: 6.0288
Epoch 3/10
101018/101018 [==============================] - 632s 6ms/step - loss: 6.0250
Epoch 4/10
101018/101018 [==============================] - 625s 6ms/step - loss: 6.0250
Epoch 5/10
101018/101018 [==============================] - 662s 7ms/step - loss: 6.0253
Epoch 6/10
101018/101018 [==============================] - 624s 6ms/step - loss: 6.0235
Epoch 7/10
101018/101018 [==============================] - 636s 6ms/step - loss: 6.0245
Epoch 8/10
101018/101018 [==============================] - 628s 6ms/step - loss: 6.0253
Epoch 9/10
101018/101018 [==============================] - 625s 6ms/step - loss: 6.0284
Epoch 10/10
101018/101018 [==============================] - 621s 6ms/step - loss: 6.0216
Epoch 1/10
101018/101018 [==============================] - 634s 6ms/step - loss: 6.0258
Epoch 2/10
101018/10

In [0]:
int_to_vocab = dict((i, c) for i, c in enumerate(vocab))


{0: u'', 1: u'.', 2: u'00seven', 3: u'0eight', 4: u'0eightsix', 5: u'0eighty', 6: u'0fivenine', 7: u'0fivesix', 8: u'0fourfour', 9: u'0fournine', 10: u'0foursix', 11: u'0fourthree', 12: u'0nineteen', 13: u'0oneeight', 14: u'0onefive', 15: u'0oneone', 16: u'0seven', 17: u'0sevenseven', 18: u'0sevensix', 19: u'0six', 20: u'0sixfive', 21: u'0sixnine', 22: u'0threenine', 23: u'0two', 24: u'0two0', 25: u'a', 26: u'ab', 27: u'abandon', 28: u'abh', 29: u'abid', 30: u'abide', 31: u'abl', 32: u'ablest', 33: u'abol', 34: u'abolit', 35: u'aborigin', 36: u'about', 37: u'abov', 38: u'abraham', 39: u'abridg', 40: u'abroad', 41: u'abrog', 42: u'absolv', 43: u'absorb', 44: u'abstain', 45: u'abstract', 46: u'absurd', 47: u'abund', 48: u'abus', 49: u'ac', 50: u'academy', 51: u'acc', 52: u'acceiv', 53: u'accel', 54: u'access', 55: u'accid', 56: u'accomac', 57: u'accommod', 58: u'accompany', 59: u'accompl', 60: u'accord', 61: u'account', 62: u'accredit', 63: u'accum', 64: u'accus', 65: u'accustom', 66: u'

In [0]:
start = numpy.random.randint(0, len(dataX)-1)
#start = len(dataX)-150
pattern = dataX[start]
print "Seed:"
print "\"", ''.join([int_to_vocab[value] for value in pattern]), "\""
result = []
# generate characters
for i in range(40):
	x = numpy.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = numpy.argmax(prediction)
	result.append(int_to_vocab[index])
	seq_in = [int_to_vocab[value] for value in pattern]
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print "\nDone."

Seed:
" toohavbeencomfromafric.ifthibesotheopofnewcountrytotheinstitutincreasthedemandforandaugthepricofslavandsodoeinfactmakslavoffreembycausthemtobebroughtfromafricandsoldintobond.buthowevthimaybeweknowtheopofnewcountrytoslaverytendtotheperpetuoftheinstitutandsodoekeepmeninslaverywhootherwwouldbefre. "
[u'the', u'', u'', u'', u'', u'', u'', u'', u'', u'', u'', u'', u'', u'', u'', u'', u'', u'', u'', u'', u'', u'', u'', u'', u'', u'', u'', u'', u'', u'', u'', u'', u'', u'', u'', u'', u'', u'', u'', u'']

Done.
